In [ ]:
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
from kaggle_datasets import KaggleDatasets
GCS_PATH = KaggleDatasets().get_gcs_path()

In [ ]:
one_hot = {'healthy': [0, 0, 0, 0, 0],
           'scab': [0, 1, 0, 0, 0],
           'scab frog_eye_leaf_spot': [0, 1, 1, 0, 0],
           'frog_eye_leaf_spot': [0, 0, 1, 0, 0],
           'rust': [0, 0, 0, 1, 0],
           'complex': [1, 0, 0, 0, 0],
           'powdery_mildew': [0, 0, 0, 0, 1],
           'rust frog_eye_leaf_spot': [0, 0, 1, 1, 0],
           'frog_eye_leaf_spot complex': [1, 0, 1, 0, 0],
           'scab frog_eye_leaf_spot complex': [1, 1, 1, 0, 0],
           'powdery_mildew complex': [1, 0, 0, 0, 1],
           'rust complex': [1, 0, 0, 1, 0]}
num_classes = 5

In [ ]:
import pandas as pd
dataframe = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
dataframe = dataframe.sample(frac=1, random_state=123)
dataframe['labels'] = dataframe['labels'].map(lambda x: one_hot[x])

In [ ]:
validation_split = 0.2
split = len(dataframe) - int(len(dataframe) * validation_split)
train_dataframe = dataframe[:split]
validation_dataframe = dataframe[split:]

In [ ]:
image_size = 224
batch_size = strategy.num_replicas_in_sync * 8

steps_per_epoch = len(train_dataframe) // batch_size
validation_steps = len(validation_dataframe) // batch_size

In [ ]:
from tensorflow.keras import layers

In [ ]:
def parse_function(filename, label):
    image_string = tf.io.read_file(GCS_PATH+'/train_images/'+filename)
    image = tf.io.decode_image(image_string, channels=3, expand_animations=False)
    image = tf.image.resize(image, (image_size, image_size))
    image = tf.cast(image, tf.float32) / 255.
    return image, label

augmentor = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomContrast(.5),
    layers.experimental.preprocessing.RandomZoom(.5, .5),
    layers.experimental.preprocessing.RandomRotation(1),
    layers.experimental.preprocessing.RandomTranslation(.25, .25)
])

def data_augmentation(images, labels):
    images = augmentor.call(images)
    return images, labels

In [ ]:
def create_dataset(dataframe, training=True, include_labels=True):
    filenames = dataframe.pop('image')
    labels = dataframe.pop('labels')
    dataset = tf.data.Dataset.from_tensor_slices((filenames.values, labels.values.tolist()))
    
    dataset = dataset.shuffle(buffer_size=len(dataframe), 
                              seed=123, reshuffle_each_iteration=False)
    dataset = dataset.map(parse_function, 
                          num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size=batch_size).cache('')

    if training:
        dataset = dataset.map(data_augmentation, 
                              num_parallel_calls=tf.data.AUTOTUNE)
    if not include_labels:
        dataset = dataset.map(lambda x, y: x, 
                              num_parallel_calls=tf.data.AUTOTUNE)
        
    return dataset.repeat().prefetch(tf.data.AUTOTUNE)

In [ ]:
train_data = create_dataset(train_dataframe.copy())
validation_data = create_dataset(validation_dataframe.copy(), training=False)

In [ ]:
def build_model():
    base_model = tf.keras.applications.MobileNet(
                        include_top=False,
                        dropout=0.5,
                        weights="imagenet",
                        input_shape=(image_size, image_size, 3))
    for layer in base_model.layers:
        layer.trainable = True

    model = tf.keras.models.Sequential([
            layers.BatchNormalization(input_shape=(image_size, image_size, 3)),
            base_model,
            layers.GlobalAveragePooling2D(),
            layers.Dropout(0.5),
            layers.Dense(512, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(num_classes, activation="sigmoid")
    ])
    return model    

In [ ]:
with strategy.scope():
    model = build_model()

In [ ]:
import tensorflow_addons

In [ ]:
with strategy.scope():
    f1_score = tensorflow_addons.metrics.FBetaScore(num_classes=num_classes, 
                                                    threshold=None, beta=0.000001,
                                                    name='f1_score')
#     kappa = tensorflow_addons.metrics.CohenKappa(num_classes=num_classes)
    confusion_matrix = tensorflow_addons.metrics.MultiLabelConfusionMatrix(num_classes=num_classes)
metrics = [confusion_matrix, f1_score, 'accuracy']

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('bestmodel_tpu.h5',
                                                monitor='val_loss',
                                                mode='auto',
                                                save_best_only=True,
                                                save_weights_only=False,
                                                verbose=1)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                 mode='auto',
                                                 factor=0.1,
                                                 min_lr=0,
                                                 patience=3,
                                                 verbose=1)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                  mode='auto',
                                                  patience=7,
                                                  verbose=1)

In [ ]:
loss = tf.keras.losses.BinaryCrossentropy()

optimizer = tf.keras.optimizers.Nadam(lr=0.0001, 
                                      beta_1=0.9, beta_2=0.999, 
                                      epsilon=1e-7)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.0001, 
                                    momentum=0.9, 
                                    nesterov=True)

In [ ]:
with strategy.scope():
    class_weight = {0: 1.,
                    1: 1.,
                    2: 10.,
                    3: 1.,
                    4: 1.}

In [ ]:
epochs = 100

callbacks = [checkpoint, reduce_lr]

model.compile(optimizer=optimizer, 
              loss=loss, 
              metrics=metrics)

history = model.fit(train_data,
                    epochs=epochs,
                    verbose=1, callbacks=callbacks,
                    validation_data=validation_data,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps,
                    class_weight=class_weight)

In [ ]:
with strategy.scope():
    f1_score = tensorflow_addons.metrics.FBetaScore(num_classes=num_classes, 
                                                    threshold=None, beta=0.000001,
                                                    name='f1_score')
metrics = [f1_score]
model.compile(optimizer=optimizer, 
              loss=loss, 
              metrics=metrics)
print(model.evaluate(validation_data.take(validation_steps)))

with strategy.scope():
    f1_score = tensorflow_addons.metrics.FBetaScore(num_classes=num_classes, 
                                                    threshold=None, beta=9999999.,
                                                    name='f1_score')
metrics = [f1_score]
model.compile(optimizer=optimizer, 
              loss=loss, 
              metrics=metrics)
print(model.evaluate(validation_data.take(validation_steps)))

In [ ]:
import matplotlib.pyplot as plt
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model.load_weights('./bestmodel_tpu.h5', by_name=True)